# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../../modules')
import hull_white as hw

import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('../../data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [6]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [ ]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [ ]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

## Respuestas

Verdadero, se puede ver que la fórmula para una put o una call sobre un bono de copún cero es exactamente la misma.



### Pregunta 1

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.

Verdadero, se puede ver que la fórmula para una put o una call sobre un bono de copún cero es exactamente la misma.



Como se mencionó enteriormente, son iguales, a continuación vemos el ejemplo para una call:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$


$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{S_Z\left(T_O\right)}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$.

### Pregunta 2

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.2%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

In [9]:
c_p = hw.CallPut.CALL
strike = 0.992
r0 = curva(0.0001)
to = 1
tb = 2
z0 = math.exp(-curva(365))  # Es la curva a un año= 365 dias
zb = math.exp(-curva(730) * 730 / 365.0)
gamma = 1.0
sigma = 0.005

In [10]:
call= hw.zcb_call_put(
    c_p, 
    strike, 
    r0, 
    to, 
    tb, 
    z0, 
    zb, 
    gamma, 
    sigma
    )

print(f"El valor de la call es {call:.8%}")

El valor de la call es 0.75620014%


### Pregunta 3

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 1282 y 2454 días.

In [12]:
from finrisk import QC_Financial_3 as Qcf

In [13]:
def get_curve_from_dataframe(yf: Qcf.QCYearFraction, wf: Qcf.QCWealthFactor,
                             df_curva: pd.DataFrame) -> Qcf.ZeroCouponCurve:
    """
    Retorna un objeto Qcf.ZeroCouponCurve. Esta función requiere que `df_curva` tenga una columna
    de nombre 'plazo' y una columna de nombre 'tasa'. Se usa interpolación lineal en la curva que
    se retorna.
    """
    plazos = Qcf.long_vec()
    tasas = Qcf.double_vec()
    for row in df_curva.itertuples():
        plazos.append(row.plazo)
        tasas.append(row.tasa)
    curva = Qcf.QCCurve(plazos, tasas)
    curva = Qcf.QCLinearInterpolator(curva)
    tipo_tasa = Qcf.QCInterestRate(0.0, yf, wf)
    curva = Qcf.ZeroCouponCurve(curva, tipo_tasa)
    return curva

In [14]:
zcc = get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

In [15]:
plazo = 2454-1282
print(f"Tasa a {plazo} días es igual a {zcc.get_rate_at(plazo):.8%}")

Tasa a 1172 días es igual a 0.08543635%


### Pregunta 4

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,044.00 (365 días después).


De esta forma la tasa utilizada para calcular el pago de un cupón entre las fechas $T > t$ de la pata flotante de un swap cámara promedio $(TNA)$ se determina a través de:

$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

In [16]:
ICP0= 10000.000
ICP365= 10044.00
d=365

TNA= round(((ICP365/ICP0)-1)*360/d,4) #Se redondea a 4 decimales

print(f"El TNA es: {TNA: .8}")

El TNA es:  0.0043


### Pregunta 5

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.

La principal diferencia es como se calcula el theta.

Por simulación de Montecarlo utilizando el modelo de Hull-White se calcula de la siguiente manera:
    
$$
\begin{equation}
\theta_t=\frac{\partial f\left(0,t\right)}{\partial t}+\gamma^*f\left(0,t\right)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp\left(-2\gamma^* t\right)\right]
\end{equation}
$$

Utilizando la medida T-forward de la siguiente forma:

Para simular, puedo utilizar las funciones del notebook 13, considerando un nuevo $\theta_t$ sea éste $\overline{\theta_t}$, dado por la siguiente fórmula:

$$
\overline{\theta_t}=\theta_t-B\left(t,T\right)\sigma^2
$$

### Pregunta 6

Considere un swap a 1Y con cupones semestrales de 1.4% (lineal). Si la tasa
cupón cero a 0.5Y es 1.3% (exp), calcule la tasa cero a 1Y.

### Pregunta 7

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .447 .

In [17]:
r0= df_curva['tasa'][0]
r_= 0.005
gamma=1.0
sigma= 0.005
dt= 1/264
n_aleatorio= 0.447

#Simulación de MC
dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
result = [(0, r0),]
gamma_dt = gamma * dt

#Cálculo de un paso de dimulación de MC
r = r0
for i in range(1,2):
    r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * n_aleatorio # Discretización de Euler
    result.append((i * dt, r))

result

# El resultado es un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.


[(0, 0.0008111102098831921), (0.003787878787878788, 0.0009645319578333419)]